# Naverwerking waterdiepte

De waterdiepte kaarten worden geinterpoleerd tussen verschillende waterstand punten. Bij HHNK worden echter watervlakken opgehoogd in het 2D om te voorkomen dat er dubbele berging ontstaat. Als dit vlak binnen een rekencel valt kan het zijn dat er water op dit vlak komt te staan. Dit willen we niet.

Dit is opgelost door zelf de rasters te maken op basis van een gpkg. De werkstappen zijn;

1. NetCDF omzetten in GPKG met max wlvl
2. Cellen selecteren waar:
    - DEM minder dan 50% van opp cel
    - meer dan 95% water
    - meer dan 99% pand
2. Max wlvl van geselecteerde cellen vervangen.
3. Wlvl/wdepth raster maken op basis van gpkg/kolom.


In [1]:
import xarray 
import threedi_raster_edits as tre
from hhnk_threedi_tools import Folders
import geopandas as gpd

from hhnk_threedi_tools.core.result_rasters.netcdf_to_gridgpkg import ThreediGrid
from hhnk_threedi_tools.core.result_rasters.calculate_raster import BaseCalculatorGPKG


#User input
folder_path = r"E:\02.modellen\23_Katvoed"
scenario = "katvoed #1 piek_ghg_T1000" #mapnaam


folder = Folders(folder_path)

dem_path = folder.model.schema_base.rasters.dem.path
dem_path = r'E:\\02.modellen\\23_Katvoed\\02_schematisation\\00_basis\\rasters/dem_katvoed_ahn3.tif'

threedi_result = folder.threedi_results.one_d_two_d[scenario]


DEBUG:threedi_raster_edits.utils.dependencies:Dependency package(name='rtree', installed_version=None, installed=False, minimum_required_version=(0, 9, 7)) not available!
DEBUG:shapely.geos:Trying `CDLL(c:\PROGRA~1\3DIMOD~1.22\bin\geos_c.dll)`
DEBUG:shapely.geos:Library path: 'c:\\PROGRA~1\\3DIMOD~1.22\\bin\\geos_c.dll'
DEBUG:shapely.geos:DLL: <CDLL 'c:\PROGRA~1\3DIMOD~1.22\bin\geos_c.dll', handle 7ff9e5cb0000 at 0x22fae7eb910>
DEBUG:fiona.env:Entering env context: <fiona.env.Env object at 0x0000022FAEAE52B0>
DEBUG:fiona.env:Starting outermost env
DEBUG:fiona.env:No GDAL environment exists
DEBUG:fiona.env:New GDAL environment <fiona._env.GDALEnv object at 0x0000022FAEAE5A00> created
DEBUG:fiona._env:GDAL_DATA found in environment.
DEBUG:fiona._env:PROJ_LIB found in environment.
DEBUG:fiona._env:Started GDALEnv: self=<fiona._env.GDALEnv object at 0x0000022FAEAE5A00>.
DEBUG:fiona.env:Updated existing <fiona._env.GDALEnv object at 0x0000022FAEAE5A00> with options {}
DEBUG:fiona.env:Entere

¯\_(ツ)_/¯ (Warning):The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


DEBUG:matplotlib:matplotlib data path: C:\PROGRA~1\3DIMOD~1.22\apps\Python39\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\wvangerwen\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'nt', 'winreg', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'ntpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', '_heapq', 'heapq', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', 'collections.abc', '_functools', 'functools', 'contextlib', 'enum

1. Klaarzetten grid

In [5]:
#Select result
threedigrid = ThreediGrid(folder=folder, threedi_result=threedi_result)

#Convert netcdf to grid gpkg
threedigrid.netcdf_to_grid_gpkg()

#Replace waterlevel of selected cells with avg of neighbours.
threedigrid.waterlevel_correction(output_col="wlvl_max_replaced")

¯\_(ツ)_/¯ (Warning):__len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
¯\_(ツ)_/¯ (Warning):__len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
¯\_(ツ)_/¯ (Warning):__len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.


2. Berekenen rasters

In [ ]:
OVERWRITE=False

grid_gdf = gpd.read_file(threedi_result.pl/"grid_corr.gpkg", driver="GPKG")


In [6]:
calculator_kwargs = {"dem_path":dem_path,
                        "grid_gdf":grid_gdf, 
                        "wlvl_column":"wlvl_max_replaced"}

#Init calculator
with BaseCalculatorGPKG(**calculator_kwargs) as self:
    self.run(output_folder=threedi_result.pl,  
                output_raster_name="wlvl_corr.tif",
                mode="MODE_WLVL",
                overwrite=OVERWRITE)
    self.run(output_folder=threedi_result.pl,  
                output_raster_name="wdepth_corr.tif",
                mode="MODE_WDEPTH",
                overwrite=OVERWRITE)
    print("Done.")

creating output raster: E:\02.modellen\23_Katvoed\03_3di_results\1d2d_results\piek_GHG_T1000\wlvl_corr.tif


¯\_(ツ)_/¯ (Warning):The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.


creating output raster: E:\02.modellen\23_Katvoed\03_3di_results\1d2d_results\piek_GHG_T1000\wdepth_corr.tif
Done.


In [7]:
calculator_kwargs = {"dem_path":dem_path,
                        "grid_gdf":grid_gdf, 
                        "wlvl_column":"wlvl_max_orig"}

with BaseCalculatorGPKG(**calculator_kwargs) as self:
    self.run(output_folder=threedi_result.pl,  
                output_raster_name="wdepth_orig.tif",
                mode="MODE_WDEPTH",
                overwrite=OVERWRITE)

creating output raster: E:\02.modellen\23_Katvoed\03_3di_results\1d2d_results\piek_GHG_T1000\wdepth_orig.tif


¯\_(ツ)_/¯ (Warning):The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.


3. Berekenen Schaderaster

In [10]:
# Schadeschatter heeft wat extra voorbereiding nodig.
from pathlib import Path
schadeschatter_path = Path(r"E:\01.basisgegevens\hhnk_schadeschatter")

import sys
if str(schadeschatter_path) not in sys.path:
    sys.path.append(str(schadeschatter_path))
import hhnk_schadeschatter as hhnk_wss


#Variables
# cfg_file = schadeschatter_path/'01_data/cfg/cfg_hhnk_2020.cfg'
cfg_file = schadeschatter_path/'01_data/cfg/cfg_lizard.cfg'
landuse_file = schadeschatter_path/'01_data/landuse2020_tiles/combined_rasters.vrt'

depth_file = threedi_result.pl/"wdepth_orig.tif"
output_file = threedi_result.pl/"damage_orig_lizard.tif"




wss_settings = {'duur_uur': 48, #uren
                'herstelperiode':'10 dagen',
                'maand':'sep',
                'cfg_file':cfg_file,
                'dmg_type':'gem'}

#Calculation
self = hhnk_wss.wss_main.Waterschadeschatter(depth_file=depth_file, 
                        landuse_file=landuse_file, 
                        output_file=output_file,
                        wss_settings=wss_settings)

# Aanmaken leeg output raster.
self.create_output_raster()

# Berkenen schaderaster
self.run(initialize_output=False)

Uitzondering - Gamma herstelperiode voor lizard config is vermenigvuldigd met herstelperiode.
creating output raster: E:\02.modellen\23_Katvoed\03_3di_results\1d2d_results\piek_GHG_T1000\damage_orig_lizard.tif


In [10]:
wss_local

NameError: name 'wss_local' is not defined